In [3]:
import tensorflow as tf

#download MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#transform images into float32 data type
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
#flattening 28*28 image into 784 dimensions
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
#normalize [0, 255] into [0, 1]
x_train, x_test = x_train / 255., x_test / 255.
#adapt one-hot encoding to lable data
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

#shuffle data using tf.data API
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

#set tf.Variable for Softmax Regression model
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))

#define Softmax Regression model
@tf.function
def softmax_regression(x):
  logits = tf.matmul(x, W) + b
  return tf.nn.softmax(logits)

#define cross-antropy loss function
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))
  #with tf.nn.softmax_cross_entropy_with_logits API
  #return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logtis, labels=y))

#define accuracy function
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

#set gradient descent optimizer for oprimize
optimizer = tf.optimizers.SGD(0.5)

#define optimization function
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    y_pred = softmax_regression(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, [W, b])
  optimizer.apply_gradients(zip(gradients, [W, b]))

#repeat 1000 times to optimize
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(batch_xs, batch_ys)

print("Accuracy: %f" % compute_accuracy(softmax_regression(x_test), y_test))

Accuracy: 0.917200
